In [ ]:
!pip -q install langchain langchain-openai langchain-community chromadb gradio

In [1]:
import os

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "AGENT_ret_2"                 # 수업용 프로젝트명

In [2]:

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage

In [9]:
# ============================================
# 4. 예시 한국어 문서 생성 (더 상세한 metadata 포함)
# ============================================
docs = [
    Document(
        page_content="이순신 장군은 임진왜란 때 조선 수군을 이끈  위대한 장군이다.",
        metadata={"source": "한국사 교과서", "page_num": 45,
"chapter": "임진왜란", "author": "국사편찬위원회"}
    ),
    Document(
        page_content="세종대왕은 한글을 창제하여 백성이 글을 쉽게  배울 수 있게 했다.",
        metadata={"source": "조선왕조실록", "page_num": 128,
"chapter": "세종실록", "author": "조선왕조실록편찬위원회"}
    ),
    Document(
        page_content="정약용은 조선 후기의 실학자로 목민심서를  저술했다.",
        metadata={"source": "실학사상사", "page_num": 203,
            "chapter": "정약용의 사상", "author": "김철수"}
    ),
]

# ============================================
# 5. 임베딩 + 벡터스토어 구성
# ============================================
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [10]:
# ============================================
# 6. Retriever Tool 정의 (상세한 metadata 포함)
# ============================================
def retrieve_qa(query: str) -> str:
    results = retriever.get_relevant_documents(query)
    formatted_results = []
    
    for r in results:
        content = r.page_content
        source = r.metadata.get("source", "알 수 없는 출처")
        page_num = r.metadata.get("page_num", "알 수 없는 페이지")
        chapter = r.metadata.get("chapter", "")
        author = r.metadata.get("author", "")
        
        # 상세한 출처 정보 구성
        citation = f"출처: {source}"
        if page_num != "알 수 없는 페이지":
            citation += f", 페이지: {page_num}"
        if chapter:
            citation += f", 장: {chapter}"
        if author:
            citation += f", 저자: {author}"
        
        formatted_result = f"내용: {content}\n{citation}"
        formatted_results.append(formatted_result)
    
    return "\n\n".join(formatted_results)

retriever_tool = Tool(
    name="retrieve_qa",
    description="한국 역사 인물에 대해 질문하면 관련 문서를 찾아준다. 문서의 내용과 함께 상세한 출처 정보(출처, 페이지, 장, 저자)도 제공한다.",
    func=retrieve_qa
)

In [11]:
# ============================================
# 7. Agent 생성
# ============================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [retriever_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 한국 역사 전문가이다. retriever 도구를 이용해  답변하라. "
    "검색된 문서에서 제공되는 출처와 페이지 정보를 반드시 포함하여  답변해라. "
    "답변 형식: [답변 내용] (출처: [source], 페이지: [page_num])"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,
verbose=True)

# ============================================
# 8. 실행 예시
# ============================================
print("=== 첫 번째 질문 ===")
print(agent_executor.invoke({"input": "한글을 만든 사람은 누구야?",
"chat_history": []})["output"])

print("\n=== 두 번째 질문 ===")
print(agent_executor.invoke({"input": "이순신에 대해 알려줘",
"chat_history": []})["output"])

print("\n=== 세 번째 질문 ===")
print(agent_executor.invoke({"input": "정약용은 어떤 일을 했나요?",
"chat_history": []})["output"])

=== 첫 번째 질문 ===


> Entering new AgentExecutor chain...

Invoking: `retrieve_qa` with `한글을 만든 사람`


내용: 세종대왕은 한글을 창제하여 백성이 글을 쉽게  배울 수 있게 했다.
출처: 조선왕조실록, 페이지: 128, 장: 세종실록, 저자: 조선왕조실록편찬위원회

내용: 세종대왕은 한글을 창제하여 백성이 글을 쉽게 배울 수 있게 했다.
출처: reg, 페이지: 2한글을 만든 사람은 세종대왕입니다. 그는 한글을 창제하여 백성이 글을 쉽게 배울 수 있도록 하였습니다. (출처: 조선왕조실록, 페이지: 128, 장: 세종실록, 저자: 조선왕조실록편찬위원회)

> Finished chain.
한글을 만든 사람은 세종대왕입니다. 그는 한글을 창제하여 백성이 글을 쉽게 배울 수 있도록 하였습니다. (출처: 조선왕조실록, 페이지: 128, 장: 세종실록, 저자: 조선왕조실록편찬위원회)

=== 두 번째 질문 ===


> Entering new AgentExecutor chain...

Invoking: `retrieve_qa` with `이순신`


내용: 이순신 장군은 임진왜란 때 조선 수군을 이끈  위대한 장군이다.
출처: 한국사 교과서, 페이지: 45, 장: 임진왜란, 저자: 국사편찬위원회

내용: 이순신 장군은 임진왜란 때 조선 수군을 이끈 위대한 장군이다.
출처: reg, 페이지: 1이순신 장군은 임진왜란 때 조선 수군을 이끈 위대한 장군입니다. (출처: 한국사 교과서, 페이지: 45, 장: 임진왜란, 저자: 국사편찬위원회)

> Finished chain.
이순신 장군은 임진왜란 때 조선 수군을 이끈 위대한 장군입니다. (출처: 한국사 교과서, 페이지: 45, 장: 임진왜란, 저자: 국사편찬위원회)

=== 세 번째 질문 ===


> Entering new AgentExecutor chain...

Invoking: `retrieve_qa` with `정약용`


내용: 정약용은 조선 

In [ ]:
# ============================================
# 9. 개선된 Agent 테스트 (metadata 정보 포함)
# ============================================

# 첫 번째 질문 테스트
print("=== 첫 번째 질문: 한글 창제 ===")
result1 = agent_executor.invoke({"input": "한글을 만든 사람은 누구야?", "chat_history": []})
print(f"답변: {result1['output']}")
print()

# 두 번째 질문 테스트  
print("=== 두 번째 질문: 이순신 장군 ===")
result2 = agent_executor.invoke({"input": "이순신에 대해 알려줘", "chat_history": []})
print(f"답변: {result2['output']}")
print()

# 세 번째 질문 테스트
print("=== 세 번째 질문: 정약용 ===")
result3 = agent_executor.invoke({"input": "정약용은 어떤 일을 했나요?", "chat_history": []})
print(f"답변: {result3['output']}")
print()

# retriever 도구 직접 테스트
print("=== Retriever Tool 직접 테스트 ===")
direct_result = retrieve_qa("세종대왕")
print("직접 검색 결과:")
print(direct_result)